In [20]:
# импортируем необходимые библиотеки
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
import numpy as np
from math import fabs

In [ ]:
# загружаем исходные данные из файла Excel с помощью метода read_excel библиотеки Pandas
df = pd.read_excel('ds_teachers.xlsx')
# удаляем дубликаты строк
df.drop_duplicates(inplace=True)
# удаляем строки, содержащие пропущенные значения
df.dropna(inplace=True)
# удаляем строки, содержащие определенные значения в столбце 'value_shortTitle'
df.drop(df[df['value_shortTitle'] == 'недсд.'].index, inplace = True)
df.drop(df[df['value_shortTitle'] == 'уваж.'].index, inplace = True)
df.drop(df[df['value_shortTitle'] == 'недоп.'].index, inplace = True)

# заменяем значения в столбце 'value_shortTitle' на числовые значения
No_val = [
    ['з', 5],
    ['н/я', 0],
    ['н/з', 2]
]
for i in No_val:
    df['value_shortTitle'] = df['value_shortTitle'].replace(i[0], i[1])

In [13]:
len(df.index)

74115

In [46]:
df

,year_title,formRd_title,regel_title,group_type,teacher,developForm,student_current_group_title,student_booknumber,course_number,value_title,value_shortTitle,Пол,Факультет,Направление подготовки
0,2017/2018,Кафедра экологической безопасности и управлени...,Биология,Контрольная работа,Соловьев С.В.,Заочная,ПР71,171158,1,Зачтено,5,М,ФГС,05.03.06 Экология и природопользование
1,2017/2018,Кафедра экологической безопасности и управлени...,Биология,Экзамен,Соловьев С.В.,Заочная,ПР71,171158,1,Хорошо,4,М,ФГС,05.03.06 Экология и природопользование
2,2017/2018,Кафедра экологической безопасности и управлени...,Геология,Контрольная работа,Катункина Е.В.,Заочная,ПР71,171158,1,Зачтено,5,М,ФГС,05.03.06 Экология и природопользование
3,2017/2018,Кафедра экологической безопасности и управлени...,Геология,Экзамен,Катункина Е.В.,Заочная,ПР71,171158,1,Хорошо,4,М,ФГС,05.03.06 Экология и природопользование
4,2017/2018,Кафедра экологической безопасности и управлени...,География,Контрольная работа,Катункина Е.В.,Заочная,ПР71,171158,1,Зачтено,5,М,ФГС,05.03.06 Экология и природопользование
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79808,2022/2023,Кафедра статистики,Профессиональные статистические программы,Зачет дифференцированный,Сумская Т.В.,Заочная,ЭП81,183606,5,Отлично,5,Ж,ФКЭиП,38.03.01 Экономика
79809,2022/2023,Кафедра статистики,Профессиональные статистические программы,Зачет дифференцированный,Сумская Т.В.,Заочная,ЭП81,180856,5,Удовлетворительно,3,Ж,ФКЭиП,38.03.01 Экономика
79810,2022/2023,Кафедра статистики,Профессиональные статистические программы,Зачет дифференцированный,Сумская Т.В.,Заочная,ЭП81,181783,5,Удовлетворительно,3,Ж,ФКЭиП,38.03.01 Экономика
79811,2022/2023,Кафедра статистики,Профессиональные статистические программы,Зачет дифференцированный,Сумская Т.В.,Заочная,ЭП81,181808,5,Удовлетворительно,3,Ж,ФКЭиП,38.03.01 Экономика


In [43]:

def score_by_teacher(df):
    # создаем новый DataFrame, копируя исходный
    df_dup = df
    # выбираем только нужные столбцы из DataFrame и сохраняем в новый DataFrame
    teacher_short_df = df[['teacher','student_booknumber','value_shortTitle']]
    # удаляем строки, содержащие точку и запятую в столбце 'teacher'
    teacher_short_df.drop(teacher_short_df[teacher_short_df['teacher'].str.contains('.,')].index, inplace = True)

    # выводим количество уникальных значений в каждом столбце нового DataFrame
    teacher_short_df.nunique()
    # преобразуем значения в столбце 'value_shortTitle' в целочисленный тип
    teacher_short_df['value_shortTitle'].astype(int)

    # получаем список преподавателей
    teacher_list = teacher_short_df['teacher'].unique()

    # создаем пустой список arr для хранения результатов
    arr = []

    # обходим список преподавателей
    for number_iter,cnt in enumerate(teacher_list):
        # фильтруем DataFrame по преподавателю cnt
        temp_df = teacher_short_df[teacher_short_df['teacher'] == cnt]
        # получаем количество уникальных студентов, оценок и среднюю оценку по преподавателю cnt
        dep_count_stud = temp_df['student_booknumber'].nunique()
        dep_count_marks = sum(temp_df['value_shortTitle'])
        dep_avg = dep_count_marks / dep_count_stud
        # добавляем запись со значениями в список arr
        arr.append([cnt,number_iter,dep_count_stud,dep_avg])

        
    # задаем названия столбцов для нового DataFrame
    colname = ['teacher', 'cat_teacher','cnt_stud', 'avg_mark']
    # создаем новый DataFrame с помощью списка arr и названий столбцов colname
    df_cnt = pd.DataFrame(arr, columns= colname)
    df_cnt
    # сливаем два DataFrame по столбцу 'teacher'
    df_feat = pd.merge(teacher_short_df, df_cnt[['teacher', 'cat_teacher','cnt_stud', 'avg_mark']], on='teacher')
    # создаем объект OrdinalEncoder для преобразования категориальных значений в числовые
    oe = OrdinalEncoder()

    # преобразуем категории в числовые значения
    encoded_cat_teacher = oe.fit_transform(df_feat[['cat_teacher']])

    # заменяем значения в столбце 'teacher' на закодированные значения
    df_feat['teacher'] = encoded_cat_teacher.flatten()
    df_feat
    # Разделяем данные на признаки (X) и целевую переменную (y)
    X = df_feat.drop('avg_mark', axis=1)
    y = df_feat['avg_mark']

    # Разделяем данные на тренировочную и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #Создаем объект линейной регрессии и обучаем его на разделенных выборках
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    #Запускаем прогнозирование по модели
    y_pred = lr.predict(X_test)
    #Оцениваем качество модели на тестовых данных, рассчитывая среднеквадратичную ошибку (mean squared error, MSE):
    mse = mean_squared_error(y_test, y_pred)
    print('MSE: ', mse)
    #Поскольку MSE показывает слишком высокие значения, необходимо подбирать гиперпараметры
    #Создаем сетку параметров
    param_grid = {
        'fit_intercept': [True, False],
        'copy_X': [True, False],
        'n_jobs': [1, -1],
        'positive': [True, False]
    }


    # Создаем объект GridSearchCV и передаем ему модель, сетку параметров и количество фолдов для кросс-валидации
    grid_search = GridSearchCV(lr, param_grid, cv=5)

    # Обучаем объект GridSearchCV на тренировочных данных
    grid_search.fit(X_train, y_train)

    # Получаем лучшую комбинацию параметров и соответствующее значение MSE
    print('Best parameters: ', grid_search.best_params_)
    print('MSE: ', grid_search.best_score_)
    #Создаем новую модель линейной регрессии по подобранным параметрам
    lr_new = LinearRegression(copy_X=True, fit_intercept=True, n_jobs= 1, positive=False)
    #Проводим обучение, тестирование модели по новым гиперпараметрам
    lr_new.fit(X_train, y_train)
    y_pred_new = lr_new.predict(X_test)
    mse_new = mean_squared_error(y_test, y_pred_new)
    print('MSE for new model: ', mse_new)
    #Значение MSE слишком высокое - отправляем на перемоделирование
    y_pred = grid_search.best_estimator_.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print('MSE on test set:', mse)



    # Создаем объект модели
    rf = RandomForestRegressor(n_estimators=100, random_state=42)

    # Обучаем модель на тренировочных данных
    rf.fit(X_train, y_train)

    # Оцениваем качество модели на тестовых данных
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print('MSE on test set:', mse)
    #MSE - тест показывает очень хорошие результаты

    #тем создается новый столбец "predicted_avg_mark" в датафрейме X_test с предсказанными средними оценками:
    X_test['predicted_avg_mark'] = rf.predict(X_test)
    print(X_test[['cat_teacher', 'cnt_stud', 'predicted_avg_mark']])
    # Добавляем столбец с предсказанными средними оценками в DataFrame df_cnt
    df_feat['predicted_avg_mark'] = rf.predict(df_feat.drop('avg_mark', axis=1))

    # Рассчитываем взвешенную сумму средних оценок для каждого преподавателя
    weighted_avg = []
    for index, row in df_feat.iterrows():
        weighted_avg.append(row['predicted_avg_mark']*row['cnt_stud'])
    df_feat['weighted_avg'] = weighted_avg
    df_feat = df_feat.sort_values('weighted_avg', ascending=False)

    # Выводим отсортированный по рейтингу DataFrame
    print(df_feat[['teacher', 'weighted_avg']])
    return df_feat[['teacher', 'weighted_avg']]

In [ ]:
def score_by_demanding_teacher(df):
    # создаем новый DataFrame, копируя исходный
    df_dup = df
    # выбираем только нужные столбцы из DataFrame и сохраняем в новый DataFrame
    teacher_short_df = df[['teacher','student_booknumber','value_shortTitle']]
    # удаляем строки, содержащие точку и запятую в столбце 'teacher'
    teacher_short_df.drop(teacher_short_df[teacher_short_df['teacher'].str.contains('.,')].index, inplace = True)

    # выводим количество уникальных значений в каждом столбце нового DataFrame
    teacher_short_df.nunique()
    # преобразуем значения в столбце 'value_shortTitle' в целочисленный тип
    teacher_short_df['value_shortTitle'].astype(int)

    # получаем список преподавателей
    teacher_list = teacher_short_df['teacher'].unique()

    # создаем пустой список arr для хранения результатов
    arr = []
    # обходим список преподавателей
    #Условимся, что требовательный преподователь - тот у кого средняя оценка ниже
    for number_iter,cnt in enumerate(teacher_list):
        # фильтруем DataFrame по преподавателю cnt
        temp_df = teacher_short_df[teacher_short_df['teacher'] == cnt]
        # получаем количество уникальных студентов, оценок и среднюю оценку по преподавателю cnt
        dep_count_stud = temp_df['student_booknumber'].nunique()
        dep_count_marks = sum(temp_df['value_shortTitle'])
        dep_avg = dep_count_marks / dep_count_stud
        # добавляем запись со значениями в список arr
        arr.append([cnt,number_iter,dep_count_stud,dep_avg])
    # задаем названия столбцов для нового DataFrame
    colname = ['teacher', 'cat_teacher','cnt_stud', 'avg_mark']
    # создаем новый DataFrame с помощью списка arr и названий столбцов colname
    df_cnt = pd.DataFrame(arr, columns= colname)

    mean_avg = df_cnt['avg_mark'].mean()
    #создаем df с выпавшими по avg значнениями
    df_outliers = df_cnt.drop(df_cnt[df_cnt['avg_mark'] < mean_avg].index)
    #создаем df с нормальными значениями
    df_normal = df_cnt.drop(df_cnt[df_cnt['avg_mark'] > mean_avg].index)
    #считаем фактор требовательности
    factor_demanding = len(df_outliers.index)/len(df_normal)
    #добавляем колонку в df_cnt с рейтингом требовательности
    df_cnt['demanding_rating'] = abs(1-df_cnt['avg_mark'])*factor_demanding/df_cnt['cnt_stud']
    # создаем объект OrdinalEncoder для преобразования категориальных значений в числовые
    df_feat = pd.merge(teacher_short_df, df_cnt[['teacher', 'cat_teacher','cnt_stud', 'demanding_rating']], on='teacher')
    oe = OrdinalEncoder()

    # преобразуем категории в числовые значения
    encoded_cat_teacher = oe.fit_transform(df_feat[['cat_teacher']])

    # заменяем значения в столбце 'teacher' на закодированные значения
    df_feat['teacher'] = encoded_cat_teacher.flatten()
 
    # Разделяем данные на признаки (X) и целевую переменную (y)
    X = df_feat.drop('demanding_rating', axis=1)
    y = df_feat['demanding_rating']

     # Разделяем данные на тренировочную и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #Создаем объект линейной регрессии и обучаем его на разделенных выборках
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    #Запускаем прогнозирование по модели
    y_pred = lr.predict(X_test)
    #Оцениваем качество модели на тестовых данных, рассчитывая среднеквадратичную ошибку (mean squared error, MSE):
    mse = mean_squared_error(y_test, y_pred)
    print('MSE: ', mse)
    #Поскольку MSE показывает слишком высокие значения, необходимо подбирать гиперпараметры
    #Создаем сетку параметров
    param_grid = {
        'fit_intercept': [True, False],
        'copy_X': [True, False],
        'n_jobs': [1, -1],
        'positive': [True, False]
    }


    # Создаем объект GridSearchCV и передаем ему модель, сетку параметров и количество фолдов для кросс-валидации
    grid_search = GridSearchCV(lr, param_grid, cv=5)

    # Обучаем объект GridSearchCV на тренировочных данных
    grid_search.fit(X_train, y_train)

    # Получаем лучшую комбинацию параметров и соответствующее значение MSE
    print('Best parameters: ', grid_search.best_params_)
    print('MSE: ', grid_search.best_score_)
    #Создаем новую модель линейной регрессии по подобранным параметрам
    lr_new = LinearRegression(copy_X=True, fit_intercept=True, n_jobs= 1, positive=False)
    #Проводим обучение, тестирование модели по новым гиперпараметрам
    lr_new.fit(X_train, y_train)
    y_pred_new = lr_new.predict(X_test)
    mse_new = mean_squared_error(y_test, y_pred_new)
    print('MSE for new model: ', mse_new)
    #Значение MSE слишком высокое - отправляем на перемоделирование
    y_pred = grid_search.best_estimator_.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print('MSE on test set:', mse)



    # Создаем объект модели
    rf = RandomForestRegressor(n_estimators=100, random_state=42)

    # Обучаем модель на тренировочных данных
    rf.fit(X_train, y_train)

    # Оцениваем качество модели на тестовых данных
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print('MSE on test set:', mse)
    #MSE - тест показывает очень хорошие результаты


    #создается новый столбец "predicted_demanding_rating" в датафрейме X_test с предсказанными средними оценками:
    X_test['predicted_demanding_rating'] = rf.predict(X_test)
    print(X_test[['cat_teacher', 'cnt_stud', 'predicted_demanding_rating']])
    # Добавляем столбец с предсказанными средними оценками в DataFrame df_cnt
    df_feat['predicted_demanding_rating'] = rf.predict(df_feat.drop('demanding_rating', axis=1))

    mean_avg_demanding = df_feat['predicted_demanding_rating'].mean()
    #создаем df с выпавшими по avg значнениями
    df_outliers_demaining = df_feat.drop(df_feat[df_feat['predicted_demanding_rating'] < mean_avg_demanding].index)


    print(df_outliers_demaining[['teacher', 'predicted_demanding_rating']])
    return df_outliers_demaining[['teacher', 'predicted_demanding_rating']]

In [ ]:
def score_by_easy_subject(df):
     # создаем новый DataFrame, копируя исходный
    df_dup = df
    # выбираем только нужные столбцы из DataFrame и сохраняем в новый DataFrame
    subject_short_df = df[['regel_title','student_booknumber','value_shortTitle']]

    # выводим количество уникальных значений в каждом столбце нового DataFrame
    subject_short_df.nunique()
    # преобразуем значения в столбце 'value_shortTitle' в целочисленный тип
    subject_short_df['value_shortTitle'].astype(int)

    # получаем список преподавателей
    subject_list = subject_short_df['regel_title'].unique()

    # создаем пустой список arr для хранения результатов
    arr = []
     # обходим список предметов
    #Условимся, что самый легкий предмет - тот, у кого самая высокая успеваемость
    for number_iter,cnt in enumerate(subject_list):
        # фильтруем DataFrame по преподавателю cnt
        temp_df = subject_short_df[subject_short_df['regel_title'] == cnt]
        # получаем количество уникальных студентов, оценок и среднюю оценку по предмету cnt
        dep_count_stud = temp_df['student_booknumber'].nunique()
        dep_count_marks = sum(temp_df['value_shortTitle'])
        dep_avg = dep_count_marks / dep_count_stud
        # добавляем запись со значениями в список arr
        arr.append([cnt,number_iter,dep_count_stud,dep_avg])
    # задаем названия столбцов для нового DataFrame
    colname = ['regel_title', 'cat_regel_title','cnt_stud', 'avg_mark']
    # создаем новый DataFrame с помощью списка arr и названий столбцов colname
    df_cnt = pd.DataFrame(arr, columns= colname)
    mean_avg = df_cnt['avg_mark'].mean()
    #создаем df с выпавшими по avg значнениями
    df_outliers = df_cnt.drop(df_cnt[df_cnt['avg_mark'] < mean_avg].index)
    #создаем df с нормальными значениями
    df_normal = df_cnt.drop(df_cnt[df_cnt['avg_mark'] > mean_avg].index)
    #считаем фактор легкости предмета
    lightness_factor = len(df_outliers.index)/len(df_normal)
    #добавляем колонку в df_cnt с рейтингом легкости
    df_cnt['lightness_rating'] = abs(1-df_cnt['avg_mark'])*lightness_factor/df_cnt['cnt_stud']
    df_feat = pd.merge(subject_short_df, df_cnt[['regel_title', 'cat_regel_title','cnt_stud', 'lightness_rating']], on='regel_title')
    
    oe = OrdinalEncoder()

    # преобразуем категории в числовые значения
    encoded_cat_teacher = oe.fit_transform(df_feat[['cat_regel_title']])

    # заменяем значения в столбце 'teacher' на закодированные значения
    df_feat['regel_title'] = encoded_cat_teacher.flatten()
 
    # Разделяем данные на признаки (X) и целевую переменную (y)
    X = df_feat.drop('lightness_rating', axis=1)
    y = df_feat['lightness_rating']
    
  # Разделяем данные на тренировочную и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #Создаем объект линейной регрессии и обучаем его на разделенных выборках
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    #Запускаем прогнозирование по модели
    y_pred = lr.predict(X_test)
    #Оцениваем качество модели на тестовых данных, рассчитывая среднеквадратичную ошибку (mean squared error, MSE):
    mse = mean_squared_error(y_test, y_pred)
    print('MSE: ', mse)
    #Поскольку MSE показывает слишком высокие значения, необходимо подбирать гиперпараметры
    #Создаем сетку параметров
    param_grid = {
        'fit_intercept': [True, False],
        'copy_X': [True, False],
        'n_jobs': [1, -1],
        'positive': [True, False]
    }


    # Создаем объект GridSearchCV и передаем ему модель, сетку параметров и количество фолдов для кросс-валидации
    grid_search = GridSearchCV(lr, param_grid, cv=5)

    # Обучаем объект GridSearchCV на тренировочных данных
    grid_search.fit(X_train, y_train)

    # Получаем лучшую комбинацию параметров и соответствующее значение MSE
    print('Best parameters: ', grid_search.best_params_)
    print('MSE: ', grid_search.best_score_)
    #Создаем новую модель линейной регрессии по подобранным параметрам
    lr_new = LinearRegression(copy_X=True, fit_intercept=True, n_jobs= 1, positive=False)
    #Проводим обучение, тестирование модели по новым гиперпараметрам
    lr_new.fit(X_train, y_train)
    y_pred_new = lr_new.predict(X_test)
    mse_new = mean_squared_error(y_test, y_pred_new)
    print('MSE for new model: ', mse_new)
    #Значение MSE слишком высокое - отправляем на перемоделирование
    y_pred = grid_search.best_estimator_.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print('MSE on test set:', mse)



    # Создаем объект модели
    rf = RandomForestRegressor(n_estimators=100, random_state=42)

    # Обучаем модель на тренировочных данных
    rf.fit(X_train, y_train)

    # Оцениваем качество модели на тестовых данных
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print('MSE on test set:', mse)
    #MSE - тест показывает очень хорошие результаты

     #создается новый столбец "predicted_lightness_rating" в датафрейме X_test с предсказанными средними оценками:
    X_test['predicted_lightness_rating'] = rf.predict(X_test)
    print(X_test[['cat_regel_title', 'cnt_stud', 'predicted_lightness_rating']])
    # Добавляем столбец с предсказанными средними оценками в DataFrame df_cnt
    df_feat['predicted_lightness_rating'] = rf.predict(df_feat.drop('lightness_rating', axis=1))

    mean_avg_lightness = df_feat['predicted_lightness_rating'].mean()
    #создаем df с выпавшими по avg значнениями
    df_outliers_demaining = df_feat.drop(df_feat[df_feat['predicted_lightness_rating'] < mean_avg_lightness].index)


    print(df_outliers_demaining[['regel_title', 'predicted_lightness_rating']])
    return df_outliers_demaining[['regel_title', 'predicted_lightness_rating']]

    
    

In [52]:
score_by_teacher(df)

/tmp/ipykernel_3160/4071278710.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teacher_short_df.drop(teacher_short_df[teacher_short_df['teacher'].str.contains('.,')].index, inplace = True)


MSE:  85.07015332432233
Best parameters:  {'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'positive': False}
MSE:  0.2108463934848035
MSE for new model:  85.07015332432233
MSE on test set: 85.07015332432233
MSE on test set: 0.038249582366955776
       cat_teacher  cnt_stud  predicted_avg_mark
110              0        44           20.181818
30717          142       174           10.362069
13889           44       211            6.696682
8317            15       254            9.637795
22914           96       189           17.957672
...            ...       ...                 ...
53614          297        78           13.846154
20941           79       145            6.937931
30380          140        46           22.956522
66011          416       115           11.652174
18937           73       420            5.135714

[13822 rows x 3 columns]
       teacher  weighted_avg
4765       5.0      19342.00
3809       5.0      19342.00
3785       5.0      19342.00
3784       5.0      

,teacher,weighted_avg
4765,5.0,19342.00
3809,5.0,19342.00
3785,5.0,19342.00
3784,5.0,19342.00
3783,5.0,19342.00
...,...,...
69107,480.0,5.16
69108,480.0,5.01
45945,241.0,3.87
45944,241.0,3.87


In [53]:
score_by_demanding_teacher(df)

/tmp/ipykernel_3160/4071278710.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teacher_short_df.drop(teacher_short_df[teacher_short_df['teacher'].str.contains('.,')].index, inplace = True)


MSE:  0.22017643458418268
Best parameters:  {'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'positive': False}
MSE:  0.08694815228070223
MSE for new model:  0.22017643458418268
MSE on test set: 0.22017643458418268
MSE on test set: 0.044019304270524584
     cat_teacher  cnt_stud  predicted_demanding_rating
73            73       420                    0.007035
415          415        23                    0.171581
392          392        34                    0.073574
278          278        46                    0.091339
400          400        81                    0.044693
..           ...       ...                         ...
238          238       110                    0.024088
409          409        55                    0.209894
25            25        33                    0.097097
265          265       177                    0.016186
132          132        61                    0.039551

[97 rows x 3 columns]
     teacher  predicted_demanding_rating
0        0.0       

,teacher,predicted_demanding_rating
0,0.0,0.221243
1,1.0,0.238677
5,5.0,0.735554
10,10.0,0.402738
38,38.0,1.173622
...,...,...
475,475.0,0.633579
477,477.0,0.564630
478,478.0,0.365672
479,479.0,0.857899


In [51]:
score_by_easy_subject(df)

MSE:  0.06006533254860806
Best parameters:  {'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'positive': False}
MSE:  0.17348120861422603
MSE for new model:  0.06006533254860806
MSE on test set: 0.06006533254860806
MSE on test set: 0.0007390383240528437
       cat_regel_title  cnt_stud  predicted_lightness_rating
69639              709        47                    0.048239
73172              862        10                    0.417744
21208               52       274                    0.014958
454                  3       497                    0.007771
19243               48       161                    0.029639
...                ...       ...                         ...
8671                13       766                    0.012949
4775                11       593                    0.005959
14723               26        82                    0.051644
28852              104       267                    0.015428
51072              368        49                    0.084418

[14823 ro

,regel_title,predicted_lightness_rating
1805,5.0,0.151936
1806,5.0,0.151936
1807,5.0,0.151936
1808,5.0,0.151936
1809,5.0,0.151936
...,...,...
74110,903.0,0.314370
74111,903.0,0.314370
74112,903.0,0.314370
74113,903.0,0.314370
